In [69]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [70]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML

In [81]:
IMAGE_SIZE = 64 # I resized the images to 64x64 to speed up training, since the autoencoder is taking long time to train because the paramaters are quite large and the model become quite large, which is not good for the training time
CHANNELS = 3
BATCH_SIZE = 32

In [110]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True
)
train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(64, 64),
    batch_size=32,
    class_mode="input"
)

Found 1506 images belonging to 3 classes.


In [82]:
train_generator.class_indices

{'Potato___Early_blight': 0, 'Potato___Late_blight': 1, 'Potato___healthy': 2}

In [83]:
class_names = list(train_generator.class_indices.keys())
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [103]:
count=0
for image_batch, label_batch in train_generator:
    print(image_batch[0])
    break

[[[0.45676422 0.4097054  0.44892108]
  [0.45902056 0.4119617  0.45117742]
  [0.45380905 0.4067502  0.44596592]
  ...
  [0.3428328  0.30919772 0.37043533]
  [0.3364372  0.30114308 0.37113547]
  [0.40980315 0.37676388 0.4344111 ]]

 [[0.4291139  0.38205504 0.42127076]
  [0.4289403  0.38188148 0.42109716]
  [0.42703217 0.37997335 0.41918904]
  ...
  [0.3381466  0.30433795 0.36696407]
  [0.33938777 0.30409366 0.3737389 ]
  [0.4132744  0.3804087  0.4373617 ]]

 [[0.4263955  0.37933668 0.42518845]
  [0.4269162  0.37985736 0.42501488]
  [0.44162905 0.39457023 0.4390335 ]
  ...
  [0.2908022  0.2587075  0.3170698 ]
  [0.35292003 0.32028875 0.38247436]
  [0.42344663 0.39159214 0.4494789 ]]

 ...

 [[0.6497005  0.6261711  0.68107307]
  [0.652068   0.6285386  0.68344057]
  [0.6515762  0.6280468  0.68294877]
  ...
  [0.57259107 0.56082636 0.63271236]
  [0.5822935  0.5705288  0.6420677 ]
  [0.58159924 0.56983453 0.6410263 ]]

 [[0.6577078  0.6341784  0.68908036]
  [0.6401587  0.6166293  0.67153126]


In [118]:
encoder = tf.keras.Sequential([
    layers.InputLayer(input_shape=(64, 64, 3)),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu')
])

decoder = tf.keras.Sequential([
    layers.InputLayer(input_shape=(512,)),
    layers.Dense(8 * 8 * 128, activation='relu'),
    layers.Reshape((8, 8, 128)),
    layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same'),
    layers.UpSampling2D((2, 2)),
    layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same'),
    layers.UpSampling2D((2, 2)),
    layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same'),
    layers.UpSampling2D((2, 2)),
    layers.Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same'),
])

autoencoder = tf.keras.Sequential([encoder, decoder])

autoencoder.compile(optimizer='adam', loss='mse')

In [116]:
autoencoder.summary()

Model: "sequential_36"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_34 (Sequential)           │ (None, 512)                 │       4,288,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_35 (Sequential)           │ (None, 64, 64, 3)           │       4,443,203 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,731,267 (33.31 MB)

 Trainable params: 8,731,267 (33.31 MB)

 Non-trainable params: 0 (0.00 B)

In [119]:
autoencoder.fit(
    train_generator,
    epochs=50,
    steps_per_epoch=len(train_generator),
    verbose=1
)

Epoch 1/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 9s 94ms/step - loss: 0.0308
Epoch 2/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 183us/step - loss: 0.0000e+00
Epoch 3/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - loss: 0.0225
Epoch 4/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 188us/step - loss: 0.0000e+00
Epoch 5/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 10s 87ms/step - loss: 0.0164
Epoch 6/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 194us/step - loss: 0.0000e+00
Epoch 7/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 105ms/step - loss: 0.0141
Epoch 8/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 171us/step - loss: 0.0000e+00
Epoch 9/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - loss: 0.0131
Epoch 10/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 122us/step - loss: 0.0000e+00
Epoch 11/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - loss: 0.0116
Epoch 12/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 115us/step - loss: 0.0000e+00
Epoch 13/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 79ms/step - loss: 0.0115
Epoch 14/50
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 162us/step - loss: 0.0000e+00
Epoch 15/50
48/48 ━━━━━━━━━━━━━━━━

In [121]:
autoencoder.save("./saved-models/autoencoder.keras")